In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

from sklearn.metrics import *
from copy import deepcopy
import matplotlib.pyplot as plt
import math


from tqdm.auto import tqdm
import seaborn as sns

from src.helpers import *
from src.visualize import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
from torch.utils.data import Dataset
import pickle

import optuna
from src.optuna_functions import * 


device = torch.device('cpu') #suposed to be cuda
dtype = torch.float32

import sys
sys.path.append("c:\\Users\\nerea\\OneDrive\\Documentos\\EPFL_MASTER\\PDM\\Project\\PyalData")
# to change for the actual path where PyalData has been cloned

c:\Users\nerea\anaconda3\envs\sinthlab\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
type = 'baseline_small'

In [3]:
path_to_data = './Data/Processed_Data'
if type == 'baseline':
        baseline_data = os.path.join(path_to_data, 'Tidy_Sansa_11_04.pkl')

        with open(baseline_data, 'rb') as fp:
                df_baseline = pickle.load(fp)


        X_train, y_train, X_val, y_val, X_test,\
        y_test, info_train, info_val, info_test = \
        train_test_split(df_baseline, train_variable = 'both_rates', \
                        target_variable = 'target_pos', num_folds = 5)
if type == 'baseline_small':
        baseline_data = os.path.join(path_to_data, 'Tidy_Sansa_13_04.pkl')

        with open(baseline_data, 'rb') as fp:
                df = pickle.load(fp)

        df_baseline = df.loc[df.type == 'BASELINE'].reset_index()

        X_train, y_train, X_val, y_val, X_test,\
        y_test, info_train, info_val, info_test = \
        train_test_split(df_baseline, train_variable = 'both_rates', \
                        target_variable = 'target_pos', num_folds = 5)
elif type == 'stim':
        stim_data = os.path.join(path_to_data, 'Tidy_Sansa_13_04.pkl')
        with open(stim_data, 'rb') as fp:
                df = pickle.load(fp)
        df_stim = df.loc[df.type == 'TONIC'].reset_index()

        X_train, y_train, X_val, y_val, X_test,\
        y_test, info_train, info_val, info_test = \
        train_test_split(df_stim, train_variable = 'both_rates', \
                        target_variable = 'target_pos', num_folds = 5)

# Test one of the folds first
fold_num = 'fold0'
fold = 0

print('We are testing the optimization method on fold ', fold)


X_train = X_train[fold_num]
X_val = X_val[fold_num]
X_test = X_test[fold_num]
y_test = y_test[fold_num]
y_train = y_train[fold_num]
y_val = y_val[fold_num]


# Specify that we want our tensors on the GPU and in float32
device = torch.device('cpu') #suposed to be cuda
dtype = torch.float32
path_to_models = './Models'

num_dim_output = y_train.shape[1]
num_features = X_train.shape[1]

seq_length = 75

# Reshape x_train to match the number of columns in the model's input layer
xx_train = X_train.reshape(X_train.shape[0] // seq_length, seq_length, X_train.shape[1])  
# Reshape y_train to match the number of neurons in the model's output layer
yy_train = y_train.reshape(y_train.shape[0] // seq_length, seq_length, y_train.shape[1])  

xx_val = X_val.reshape(X_val.shape[0] // seq_length, seq_length, X_val.shape[1])  
yy_val = y_val.reshape(y_val.shape[0] // seq_length, seq_length, y_val.shape[1])  

xx_test = X_test.reshape(X_test.shape[0] // seq_length, seq_length, X_test.shape[1])  
yy_test = y_test.reshape(y_test.shape[0] // seq_length, seq_length, y_test.shape[1])  

seed = 42
torch.manual_seed(seed)

Reg = globals().get(Regularizer_LSTM)


Test trials  4
Val trials 3
We are testing the optimization method on fold  0


In [4]:
# Fit the LSTM model
def train_model_optuna(trial):
    
    X = xx_train
    Y = yy_train
    X_val = xx_val
    Y_val = yy_val

    num_epochs= 1000
    early_stop = 5

    model = CausalTemporalLSTM_Optuna(trial, num_features= num_features, 
                out_dims = num_dim_output).to(device)
    
    # Set up the optimizer with the specified learning rate
    #optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer_name = 'Adam'
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log = True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr = lr)

    lr_step_size = 10 # trial.suggest_int('lr_step_size', 5, 15)
    lr_gamma = trial.suggest_float('lr_gamma', 1, 1.5)
    # Set up a learning rate scheduler
    scheduler = lr_scheduler.StepLR(optimizer, 
                                    step_size=lr_step_size, 
                                    gamma=lr_gamma)
        
    # Keep track of the best model's parameters and loss
    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1e8

    # Enable anomaly detection for debugging
    torch.autograd.set_detect_anomaly(True)

    # Track the train and validation loss
    train_losses = []
    val_losses = []
    # Counters for early stopping
    not_increased = 0
    end_train = 0
    
    # Reshape data for the LSTM
    seq_length_LSTM = trial.suggest_int('seq_length_LSTM', 5, 20)
    train_dataset = SequenceDataset(Y,X,seq_length_LSTM)
    val_dataset = SequenceDataset(Y_val,X_val,seq_length_LSTM)


    batch_size_train = trial.suggest_int('batch_size_train', 20, 50)
    batch_size_val = trial.suggest_int('batch_size_val',20, 50)
    loader_train = data.DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
    loader_val = data.DataLoader(val_dataset, batch_size=batch_size_val, shuffle=True)

    # hyperparameter for huber loss
    delta = trial.suggest_int('huber_delta', 5, 10) #was 8 before

    # hyperparameter for regularizer
    alpha = trial.suggest_float('alpha_reg', 1e-7, 1e-3, log = True) # was 1e-5 before
    l1_ratio =trial.suggest_float('l1_ratio_reg', 0.2, 0.8) #was 0.5 before

    # Loop through epochs
    for epoch in np.arange(num_epochs):
        for phase in ['train', 'val']:
            # set model to train/validation as appropriate
            if phase == 'train':
                model.train()
                loader = loader_train
            else:
                model.eval()
                loader = loader_val

            # Initialize variables to track loss and batch size
            running_loss = 0
            running_size = 0        

            # Iterate over batches in the loader
            for X_, y_ in loader:
                #X_ = X_.to('cuda')
                #y_ = y_.to('cuda')
                if phase == "train":
                    with torch.set_grad_enabled(True):
                        optimizer.zero_grad()

                        output_t = model(X_)
                        output_t = torch.squeeze(output_t)


                        loss_t = huber_loss(output_t, y_, delta)
                        
                        # Add regularization to the loss in the training phase
                        loss_t += Regularizer_LSTM(model, l1_ratio, alpha)

                        # Compute gradients and perform an optimization step
                        loss_t.backward()
                        optimizer.step()
                else:
                    # just compute the loss in validation phase
                    output_t = model(X_)
                    output_t = torch.squeeze(output_t)

                    loss_t = huber_loss(output_t, y_, delta)

                # Ensure the loss is finite
                assert torch.isfinite(loss_t)
                running_loss += loss_t.item()
                running_size += 1

            # compute the train/validation loss and update the best
            # model parameters if this is the lowest validation loss yet
            running_loss /= running_size
            if phase == "train":
                train_losses.append(running_loss)
            else:
                val_losses.append(running_loss)
                
                # Update best model parameters if validation loss improves
                if running_loss < best_loss:
                    best_loss = running_loss
                    best_model_wts = deepcopy(model.state_dict())
                    not_increased = 0
                else:
                    # Perform early stopping if validation loss doesn't improve
                    if epoch > 10:
                        not_increased += 1
                        # print('Not increased : {}/5'.format(not_increased))
                        if not_increased == early_stop:
                            print('Decrease LR')
                            for g in optimizer.param_groups:
                                g['lr'] = g['lr'] / 2
                            not_increased = 0
                            end_train += 1
                        
                        if end_train == 2:
                            model.load_state_dict(best_model_wts)
                            y_true_val, y_pred_val = reshape_to_eval(xx_val,yy_val, model)
                            ev_val = explained_variance_score(y_true_val, y_pred_val)
                            return ev_val  # here change to not return a list but a single value for the trial to analyze
                        

        # Update learning rate with the scheduler
        scheduler.step()
        print("Epoch {:03} Train {:.4f} Val {:.4f}".format(epoch, train_losses[-1], val_losses[-1]))


        y_true_val_epoch, y_pred_val_epoch = reshape_to_eval(xx_val,yy_val, model)
        ev_val_epoch = explained_variance_score(y_true_val_epoch, y_pred_val_epoch)
        trial.report(ev_val_epoch, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    y_true_val, y_pred_val = reshape_to_eval(xx_val,yy_val, model)
    ev_val = explained_variance_score(y_true_val, y_pred_val)

    return ev_val


In [5]:
def reshape_to_eval(x,y, model):
    to_t_eval =  lambda array: torch.tensor(array, device='cpu', dtype=dtype)  
    x = to_t_eval(x) 
    y = to_t_eval(y)
    y_pred = model(x)
    y_array = y.detach().cpu().numpy()
    y_pred_array = y_pred.detach().cpu().numpy()

    # Reshape tensors to 2D arrays (flatten the batch and sequence dimensions)
    y_pred_2D = y_pred_array.reshape(-1, y_pred_array.shape[-1])
    y_true_2D = y_array.reshape(-1, y_array.shape[-1])
    
    return y_true_2D, y_pred_2D

In [6]:
study = optuna.create_study(direction="maximize")

# Set pruning options
study.pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5)

study.optimize(train_model_optuna, n_trials=30)

importance_scores = optuna.importance.get_param_importances(study)

# Print importance scores
for param, score in importance_scores.items():
    print(f"{param}: {score}")

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print('Study statistics: ')
print("Number of finished trials: ", len(study.trials))
print("Number of pruned trials: ", len(pruned_trials))
print("Number of complete trials: ", len(complete_trials))

print('Best trial: ')
trial = study.best_trial

print("Loss: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2024-04-11 11:20:38,747] A new study created in memory with name: no-name-049ee5d8-a443-4910-8527-e21ed2432d16


Epoch 000 Train 0.0366 Val 0.0474
Epoch 001 Train 0.0358 Val 0.0466
Epoch 002 Train 0.0352 Val 0.0460
Epoch 003 Train 0.0345 Val 0.0454
Epoch 004 Train 0.0335 Val 0.0447
Epoch 005 Train 0.0328 Val 0.0440
Epoch 006 Train 0.0318 Val 0.0431
Epoch 007 Train 0.0308 Val 0.0420
Epoch 008 Train 0.0302 Val 0.0410
Epoch 009 Train 0.0292 Val 0.0400
Epoch 010 Train 0.0279 Val 0.0384
Epoch 011 Train 0.0271 Val 0.0365
Epoch 012 Train 0.0256 Val 0.0350
Epoch 013 Train 0.0248 Val 0.0336
Epoch 014 Train 0.0242 Val 0.0325
Epoch 015 Train 0.0233 Val 0.0313
Epoch 016 Train 0.0225 Val 0.0302
Epoch 017 Train 0.0219 Val 0.0296
Epoch 018 Train 0.0215 Val 0.0290
Epoch 019 Train 0.0210 Val 0.0284
Epoch 020 Train 0.0207 Val 0.0279
Epoch 021 Train 0.0202 Val 0.0271
Epoch 022 Train 0.0198 Val 0.0264
Epoch 023 Train 0.0191 Val 0.0259
Epoch 024 Train 0.0190 Val 0.0255
Epoch 025 Train 0.0183 Val 0.0250
Epoch 026 Train 0.0178 Val 0.0248
Epoch 027 Train 0.0176 Val 0.0246
Epoch 028 Train 0.0175 Val 0.0241
Epoch 029 Trai

[I 2024-04-11 11:22:14,798] Trial 0 finished with value: 0.557564357916514 and parameters: {'n_hidden_units': 27, 'num_layers': 2, 'input_size_LSTM': 42, 'dropout_1': 0.10860994498666071, 'dropout_2': 0.890047655818418, 'lr': 5.993393416205095e-05, 'lr_gamma': 1.3369191702379997, 'seq_length_LSTM': 9, 'batch_size_train': 26, 'batch_size_val': 47, 'huber_delta': 9, 'alpha_reg': 1.3167292982528577e-07, 'l1_ratio_reg': 0.7591944148023158}. Best is trial 0 with value: 0.557564357916514.


Decrease LR
Epoch 000 Train 0.0445 Val 0.0531
Epoch 001 Train 0.0439 Val 0.0533
Epoch 002 Train 0.0432 Val 0.0510
Epoch 003 Train 0.0427 Val 0.0506
Epoch 004 Train 0.0419 Val 0.0497
Epoch 005 Train 0.0407 Val 0.0482
Epoch 006 Train 0.0395 Val 0.0466
Epoch 007 Train 0.0374 Val 0.0432
Epoch 008 Train 0.0349 Val 0.0399
Epoch 009 Train 0.0321 Val 0.0373
Epoch 010 Train 0.0291 Val 0.0343
Epoch 011 Train 0.0268 Val 0.0326
Epoch 012 Train 0.0250 Val 0.0309
Epoch 013 Train 0.0234 Val 0.0298
Epoch 014 Train 0.0223 Val 0.0282
Epoch 015 Train 0.0213 Val 0.0274
Epoch 016 Train 0.0207 Val 0.0265
Epoch 017 Train 0.0201 Val 0.0263
Epoch 018 Train 0.0197 Val 0.0261
Epoch 019 Train 0.0192 Val 0.0249
Epoch 020 Train 0.0190 Val 0.0248
Epoch 021 Train 0.0185 Val 0.0245
Epoch 022 Train 0.0183 Val 0.0246
Epoch 023 Train 0.0178 Val 0.0240
Epoch 024 Train 0.0175 Val 0.0242
Epoch 025 Train 0.0174 Val 0.0241
Epoch 026 Train 0.0171 Val 0.0233
Epoch 027 Train 0.0169 Val 0.0231
Epoch 028 Train 0.0167 Val 0.0224
Ep

[I 2024-04-11 11:23:43,921] Trial 1 finished with value: 0.5837963223457336 and parameters: {'n_hidden_units': 40, 'num_layers': 2, 'input_size_LSTM': 22, 'dropout_1': 0.5919414109816071, 'dropout_2': 0.3992216410596555, 'lr': 4.7842834520663247e-05, 'lr_gamma': 1.1973038927346897, 'seq_length_LSTM': 17, 'batch_size_train': 30, 'batch_size_val': 24, 'huber_delta': 6, 'alpha_reg': 1.4879497663262762e-05, 'l1_ratio_reg': 0.7886856630371724}. Best is trial 1 with value: 0.5837963223457336.


...

Decrease LR
Epoch 000 Train 0.0300 Val 0.0312
Epoch 001 Train 0.0172 Val 0.0261
Epoch 002 Train 0.0130 Val 0.0211
Epoch 003 Train 0.0115 Val 0.0191
Epoch 004 Train 0.0107 Val 0.0192
Epoch 005 Train 0.0102 Val 0.0189
Epoch 006 Train 0.0099 Val 0.0188
Epoch 007 Train 0.0091 Val 0.0171
Epoch 008 Train 0.0085 Val 0.0176
Epoch 009 Train 0.0081 Val 0.0187
Epoch 010 Train 0.0080 Val 0.0228
Epoch 011 Train 0.0073 Val 0.0240
Epoch 012 Train 0.0072 Val 0.0273
Epoch 013 Train 0.0068 Val 0.0191
Epoch 014 Train 0.0065 Val 0.0243
Decrease LR
Epoch 015 Train 0.0061 Val 0.0236
Epoch 016 Train 0.0060 Val 0.0245
Epoch 017 Train 0.0057 Val 0.0257
Epoch 018 Train 0.0056 Val 0.0217
Epoch 019 Train 0.0056 Val 0.0249


[I 2024-04-11 11:43:31,175] Trial 28 finished with value: 0.6524635553359985 and parameters: {'n_hidden_units': 23, 'num_layers': 4, 'input_size_LSTM': 35, 'dropout_1': 0.8343563553295378, 'dropout_2': 0.36332051749526667, 'lr': 0.0023534103635508693, 'lr_gamma': 1.2938970141655959, 'seq_length_LSTM': 6, 'batch_size_train': 22, 'batch_size_val': 47, 'huber_delta': 6, 'alpha_reg': 2.830051228986696e-07, 'l1_ratio_reg': 0.5480711032184736}. Best is trial 25 with value: 0.6844515204429626.


Decrease LR
Epoch 000 Train 0.0360 Val 0.0477
Epoch 001 Train 0.0344 Val 0.0467
Epoch 002 Train 0.0324 Val 0.0453
Epoch 003 Train 0.0304 Val 0.0447
Epoch 004 Train 0.0280 Val 0.0425


[I 2024-04-11 11:43:45,425] Trial 29 pruned. 


Epoch 005 Train 0.0255 Val 0.0393
input_size_LSTM: 0.20705422676930574
lr_gamma: 0.15442104514456614
batch_size_val: 0.14912654395131678
seq_length_LSTM: 0.09927420071813813
dropout_1: 0.08656502788544601
l1_ratio_reg: 0.06026082337755853
dropout_2: 0.059570603141598726
lr: 0.04685162057922414
batch_size_train: 0.04351582474001469
n_hidden_units: 0.04143269986718854
huber_delta: 0.03565514817560367
num_layers: 0.01422314637692287
alpha_reg: 0.0020490892731159114
Study statistics: 
Number of finished trials:  30
Number of pruned trials:  9
Number of complete trials:  21
Best trial: 
Loss: 0.6844515204429626
Best hyperparameters: {'n_hidden_units': 29, 'num_layers': 4, 'input_size_LSTM': 31, 'dropout_1': 0.8345976574442866, 'dropout_2': 0.6125416027383583, 'lr': 0.0019300357710035122, 'lr_gamma': 1.4370057715432787, 'seq_length_LSTM': 6, 'batch_size_train': 23, 'batch_size_val': 49, 'huber_delta': 8, 'alpha_reg': 1.2105872414087716e-06, 'l1_ratio_reg': 0.39395238042273817}


In [7]:
import joblib
import json
# Retrieve the best trial
best_trial = study.best_trial

# Get the best hyperparameters
best_params = best_trial.params

# Save the best parameters
with open('stim_0411_best_params.json', 'w') as f:
    json.dump(best_params, f)

### Plotting the study

In [8]:
optuna.visualization.plot_optimization_history(study)

In [9]:
#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

In [10]:
# Plotting the optimization history of the study.

optuna.visualization.plot_optimization_history(study)

#Plotting the accuracies for each hyperparameter for each trial.
    
optuna.visualization.plot_slice(study)

# Plotting the accuracy surface for the hyperparameters involved in the random forest model.

optuna.visualization.plot_contour(study, params=["seq_length_LSTM", "n_hidden_units"]) 